<h3> Face Detection on OpenCV DNN</h3>

In [1]:
import cv2
from time import time

# Create face detector object: must not be in function to prevent repeated object creation
opencv_dnn_model = cv2.dnn.readNetFromCaffe(prototxt="./models/deploy.prototxt", caffeModel="./models/res10_300x300_ssd_iter_140000.caffemodel")

'''
This function performs face(s) detection on an image using opencv deep learning based face detector.
Args:
    image:               The input image of the person(s) whose face needs to be detected.
    opencv_dnn_model:    The pre-trained opencv deep learning based face detection model loaded from the disk
                         required to perform the detection.
    min_confidence:      The minimum detection confidence required to consider the face detection model's
                         prediction correct.
                         
Returns:
    output_image: A copy of input image with the bounding boxes drawn and confidence scores written.
    results:      The output of the face detection process on the input image.
'''
def cvDnnDetectFaces(image, opencv_dnn_model, min_confidence=0.5):

    # Get the height and width of the input image.
    image_height, image_width, _ = image.shape

    # Create a copy of the input image to draw bounding boxes and write confidence scores.
    output_image = image.copy()

    # Perform the required pre-processings on the image and create a 4D blob from image.
    # Resize the image and apply mean subtraction to its channels
    # Also convert from BGR to RGB format by swapping Blue and Red channels.
    preprocessed_image = cv2.dnn.blobFromImage(image, scalefactor=1.0, size=(300, 300),
                                               mean=(104.0, 117.0, 123.0), swapRB=False, crop=False)

    # Set the input value for the model.
    opencv_dnn_model.setInput(preprocessed_image)

    # Get the current time before performing face detection.
    start = time()

    # Perform the face detection on the image.
    results = opencv_dnn_model.forward()

    # Get the current time after performing face detection.
    end = time()

    # Loop through each face detected in the image.
    for face in results[0][0]:

        # Retrieve the face detection confidence score.
        face_confidence = face[2]

        # Check if the face detection confidence score is greater than the thresold.
        if face_confidence > min_confidence:

            # Retrieve the bounding box of the face.
            bbox = face[3:]

            # Retrieve the bounding box coordinates of the face and scale them according to the original size of the image.
            x1 = int(bbox[0] * image_width)
            y1 = int(bbox[1] * image_height)
            x2 = int(bbox[2] * image_width)
            y2 = int(bbox[3] * image_height)

            # Draw a bounding box around a face on the copy of the image using the retrieved coordinates.
            cv2.rectangle(output_image, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=image_width // 200)

            # Draw a filled rectangle near the bounding box of the face.
            # We are doing it to change the background of the confidence score to make it easily visible.
            # cv2.rectangle(output_image, pt1=(x1, y1 - image_width // 20), pt2=(x1 + image_width // 16, y1),
            #               color=(0, 255, 0), thickness=-1)

            # Write the confidence score of the face near the bounding box and on the filled rectangle.
            cv2.putText(output_image, text=str(round(face_confidence, 2)), org=(x1, y1 - 25),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=image_width // 600,
                        color=(255, 255, 0), thickness=image_width // 200)

    # Return the output image and results of face detection.
    return output_image, results


start_time = 0
capture = cv2.VideoCapture(0)

while (capture.isOpened()):

    success, frame = capture.read()

    # Check if frame is not read properly then continue to the next iteration to read the next frame.
    if not success:
        continue

    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)

    # Get the height and width of the frame.
    frame_height, frame_width, _ = frame.shape
    # Check if the OpenCV DNN algorithm is selected.

    # Perform face detection using the OpenCV DNN algorithm.
    frame, _ = cvDnnDetectFaces(frame, opencv_dnn_model)

    # Write the currently selected method on the frame.
    # cv2.putText(frame, (frame_width // 3, frame_height // 8), cv2.FONT_HERSHEY_PLAIN, 4, (255, 155, 0), 3)

    # Set the time for this frame to the current time.
    end_time = time()

    # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
    if (end_time - start_time) > 0:
        # Calculate the number of frames per second.
        frames_per_second = 1.0 / (end_time - start_time)

        # Write the calculated number of frames per second on the frame.
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0),
                    3)

    # Update the previous frame time to this frame time.
    # As this frame will become previous frame in next iteration.
    start_time = end_time

    cv2.imshow('OpenCV DNN', frame)

    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()


: 